# Walking
The purpose of this notebook is to to refine a policy that has learned to move by trotting around into a policy that wlaks properly.

As usual the directory will contain all the necessary for the execution of this notebook.

In [1]:
import time
import numpy as np
from stable_baselines3 import PPO

import sys
import os

from spotmicro.env.spotmicro_env import SpotmicroEnv
from reward_function import reward_function, RewardState

pybullet build time: Sep 28 2025 16:57:08


#### Training

In [ ]:
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.logger import configure

# ========= CONFIG ==========
TOTAL_STEPS = 16_000_000
run = "walk"
base="trot"

log_dir = f"./logs/{run}"

def clipped_linear_schedule(initial_value, min_value=1e-5):
    def schedule(progress_remaining):
        return max(progress_remaining * initial_value, min_value)
    return schedule

checkpoint_callback = CheckpointCallback(
    save_freq=TOTAL_STEPS // 16,
    save_path=f"{run}_checkpoints",
    name_prefix=f"ppo_{run}"
)

# ========= ENV ==========
env = SpotmicroEnv(
    use_gui=False,
    reward_fn=reward_function, 
    reward_state=RewardState(),
    src_save_file=f"{base}.pkl",
    dest_save_file=f"{run}.pkl"
)
check_env(env, warn=True)

# ========= MODEL ==========
model = PPO.load(f"ppo_{base}", device = 'cpu')
model.set_env(env)
model.tensorboard_log = log_dir

# Custom logger: ONLY csv + tensorboard (no stdout table)
new_logger = configure(log_dir, ["csv", "tensorboard"])
model.set_logger(new_logger)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./logs

In [ ]:
model.learn(
    total_timesteps=TOTAL_STEPS,
    reset_num_timesteps=False,
    callback=checkpoint_callback
)
model.save(f"ppo_{run}")
env.close()

# Evaluation

In [2]:
policy = "walk"

env = SpotmicroEnv(
    use_gui=True, 
    reward_fn=reward_function,
    reward_state=RewardState(),
    #src_save_file=f"{policy}.pkl"
    )
obs, _ = env.reset()

# === Load model ===
#model = PPO.load(f"ppo_{policy}")
model = PPO.load(f"{policy}_checkpoints/ppo_{policy}_23002176_steps", device = 'cpu')

# === Run rollout ===
for _ in range(3001):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        print("Terminated")
        env.plot_reward_components()
        obs, _ = env.reset()
    time.sleep(1/60)

env.close()

ridges coefficient: 0.25
Terminated
Terminated
Terminated


X connection to :1 broken (explicit kill or server shutdown).


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Intel
GL_RENDERER=Mesa Intel(R) HD Graphics 630 (KBL GT2)
GL_VERSION=4.6 (Core Profile) Mesa 25.0.7-0ubuntu0.24.04.2
GL_SHADING_LANGUAGE_VERSION=4.60
pthread_getconcurrency()=0
Version = 4.6 (Core Profile) Mesa 25.0.7-0ubuntu0.24.04.2
Vendor = Intel
Renderer = Mesa Intel(R) HD Graphics 630 (KBL GT2)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Intel
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu
ven = Intel
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu
b3

RuntimeError: unknown parameter type

# Results:
- n1 was an utter disaster, the gait moved alongisde the ridges (and so perpendicularly to the target direction). Maybe decrease forward reward AND target velocity